## Segmentation_Population

Dans ce TP, nous allons créer un modèle de clustering de la population américaine.
<br/>Le but sera de segmenter le territoire américain
<br/> <b>Intérêt</b> : Quadriller le territoire afin de comprendre où implanter une nouvelle banque. 

##### 1. Description des fichiers

<br/>Finance.csv : Fichier contenant toutes les informations relatives aux revenues.
<br/>us.2014.19ages.txt : Fichier contenant toutes les informations relatives à la distribution de la population américaine
<br/>national_county.txt
<br/>Education.xls: Niveau d'études/éducation de la population
<br/>Unemployment.xls : Niveau de chômage dans la population

##### 2.Questions

(i) Créer le dataframe qui va permettre de faire le clustering:
<br/>a-Importer et retraiter toutes les données
<br/>b-Merger et sélectionner les variables discriminantes
<br/><b>Cette question va être corrigé</b>

(ii) Faire le clustering et représenter le clustering par une ACP:
<br/>a-Faire un K-means en faisant varier le K
<br/>b-Faire une ACP
<br/>c-Représenter avec plotly les résultats du clustering avec l'ACP
<br/><b>Cette question va être corrigé</b>

(iii) Prenez la question au choix:
<br/>a-Calculer la matrice de corrélation afin de définir (si il y en a) des variables qu'on pourrait exclure afin de refaire le clustering, refaire le clustering en m'expliquant le choix du K et analyser chacun des clusters (distribution des variables par exemple, interprétation etc...)
<br/>b-Utiliser les données df_population pour essayer de prédire par exemple la variable <b>mean|family_income_mean </b> du dataframe <b> df_cluster_US_Population </b> 
</br>=> le modèle pourra être xgboost ou random forest ou celui que vous souhaitez. Vous essaierez de transformer les valeurs d'income en un modèle à 2 classe (supérieur à la médiane ou inférieur par exemple) afin d'avoir à prédire un modèle binaire. (la fonction de transformation vous sera transmise) 

Tous les imports qui seront nécessaire

In [103]:
import pandas as pd
import plotly.plotly as py
import plotly
import numpy as np
import codecs
import gc
import xgboost

<b>Lecture du fichier qui servira de source principale </b>

## Transformation des données sur la répartition de la population

In [79]:
# Lecture des données relative à la population américaine. Le fichier est en fixe delimited donc il faut indiquer la taille des chaines de caractère et le nom des colonnes qui y sont associé
df_population = pd.read_fwf('D:/Formation/Dauphine/TP1/us.2014.19ages.txt', widths=[4,2,2,3,2,1,1,1,2,8], names=['Year','State_postal_abbreviation','State_FIPS_code','County_FIPS','code','Race','Origin','Sex','Age','Population'], dtype=str)
df_population['FIPS_code'] = df_population['State_FIPS_code'] + df_population['County_FIPS']

# Le dictionnaire de données qui traduit les valeurs
d_ethnicity = {'Race': ['1', '2', '3', '4'], 'Race_Lib': ['White', 'Black', 'American Indian/Alaska Native', 'Asian or Pacific Islander']}
df_ethnicity = pd.DataFrame(data=d_ethnicity)
d_origin = {'Origin': ['0', '1', '9'], 'Origin_Lib': ['Non-Hispanic', 'Hispanic', 'Not applicable']}
df_origin = pd.DataFrame(data=d_origin)
d_age = {'Age': ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18'], 'Age_Lib': ['0 Years', '1 - 4 Years', '5 - 9 Years' , '10 - 14 Years', '15 - 19 Years', '20 - 24 Years',
'25 - 29 Years', '30 - 34 Years', '35 - 39 Years', '40 - 44 Years', '45 - 49 Years', '50 - 54 Years', '55 - 59 Years', '60 - 64 Years',
'65 - 69 Years', '70 - 74 Years', '75 - 79 Years', '80 - 84 Years', '85+ Years'],
'Age_Lib_bins': ['0 - 20 Years', '0 - 20 Years', '0 - 20 Years' , '0 - 20 Years', '0 - 20 Years', '20 - 45 Years',
'20 - 45 Years', '20 - 45 Years', '20 - 45 Years', '20 - 45 Years', '45 - 65 Years', '45 - 65 Years', '45 - 65 Years', '45 - 65 Years',
'65+ Years', '65+ Years', '65+ Years', '65+ Years', '65+ Years']}
df_age = pd.DataFrame(data=d_age)
d_sex = {'Sex': ['1', '2'], 'Sex_Lib': ['Male', 'Female']}
df_sex = pd.DataFrame(data=d_sex)

In [80]:
# Jointure avec les différents dictionnaires
df_population = df_population.merge(df_ethnicity, on= 'Race')
df_population = df_population.merge(df_origin, on='Origin')
df_population = df_population.merge(df_sex, on='Sex')
df_population = df_population.merge(df_age, on='Age')

In [81]:
# Calcul de la population totale par FIPS_CODE et de la population relative par segment
df_population.Population = df_population.Population.astype(float)  
df_population_Total_fips = pd.pivot_table(df_population, values='Population', index=['FIPS_code','Year'], aggfunc=np.sum)
df_population_Total_fips = df_population_Total_fips.reset_index()
df_population_Total_fips = df_population_Total_fips.rename(columns={'Population':'Population_Totale'})
df_population['FIPS_code'] = df_population['FIPS_code'].astype(str)
df_population = df_population.merge(df_population_Total_fips, on=['Year','FIPS_code'])
df_population['Population_Relative'] = df_population.Population / df_population.Population_Totale 

In [82]:
# df_population = pd.pivot_table(df_population, values=['Population_Relative','Population'], columns=['Race_Lib','Sex_Lib','Age_Lib_bins'], index=['Year','State_postal_abbreviation','FIPS_code'], aggfunc=np.sum) #'Origin_Lib',
# Calcul du dataset finale avec un pivot pr avoir une ligne par FIPS
df_population = pd.pivot_table(df_population, values='Population_Relative', columns=['Race_Lib','Sex_Lib','Age_Lib_bins'], index=['Year','State_postal_abbreviation','FIPS_code'], aggfunc=np.sum)
df_population = df_population.fillna(0)
df_population.columns = ["|".join((j,k,m)) for j,k,m in df_population.columns]
df_population = df_population.reset_index()
df_population = df_population.merge(df_population_Total_fips, on=['Year','FIPS_code'])
# Aggrégation des colonnes relatives au natives américain car la population aujourd'hui est trop faible (cela permet de réduire le nombre de variable pour le clustering)
df_population['Population_Relative_Native_Female'] = df_population['American Indian/Alaska Native|Female|0 - 20 Years'] + df_population['American Indian/Alaska Native|Female|20 - 45 Years'] + df_population['American Indian/Alaska Native|Female|45 - 65 Years'] + df_population['American Indian/Alaska Native|Female|65+ Years']
df_population['Population_Relative_Native_Male'] = df_population['American Indian/Alaska Native|Male|0 - 20 Years'] + df_population['American Indian/Alaska Native|Male|20 - 45 Years'] + df_population['American Indian/Alaska Native|Male|45 - 65 Years'] + df_population['American Indian/Alaska Native|Male|65+ Years']
df_population = df_population.drop(columns=['American Indian/Alaska Native|Female|0 - 20 Years',
       'American Indian/Alaska Native|Female|20 - 45 Years',
       'American Indian/Alaska Native|Female|45 - 65 Years',
       'American Indian/Alaska Native|Female|65+ Years',
       'American Indian/Alaska Native|Male|0 - 20 Years',
       'American Indian/Alaska Native|Male|20 - 45 Years',
       'American Indian/Alaska Native|Male|45 - 65 Years',
       'American Indian/Alaska Native|Male|65+ Years'])

#### Niveau d'éducation par FIPS

In [83]:
df_Education = pd.read_excel('D:/Formation/Dauphine/TP1/Education.xls', dtype=str)
df_Education= df_Education[['FIPS Code',
       "Percent of adults with less than a high school diploma, 2010-2014",
       "Percent of adults with a high school diploma only, 2010-2014",
       "Percent of adults completing some college or associate's degree, 2010-2014",
       "Percent of adults with a bachelor's degree or higher, 2010-2014"]]
df_population = df_population.merge(df_Education, left_on='FIPS_code', right_on='FIPS Code', how='inner')

#### Taux de chômage par FIPS

In [84]:
df_Unemployment = pd.read_excel('D:/Formation/Dauphine/TP1/Unemployment.xls', dtype=str, skiprows=6)
df_Unemployment = df_Unemployment[['FIPS_Code', 'State', 'Unemployment_rate_2014']]
df_population = df_population.merge(df_Unemployment, left_on='FIPS_code', right_on='FIPS_Code', how='inner')

<b>Merge de la population totale <b>

In [ ]:
df_population['Unemployment_rate_2014'] = df_population['Unemployment_rate_2014'].astype(float) 
df_population['Unemployment_rate_2014'] = df_population['Unemployment_rate_2014'] / 100

In [85]:
# Save intermédiaire du dataset finale
df_population.to_csv('D:/Formation/Dauphine/TP1/Population_US_2014_rel.csv', sep=';', decimal='.')

In [89]:
dataset = df_population.drop(columns=['FIPS_Code', 'State', 'State_postal_abbreviation', 'FIPS_code','Year','FIPS Code','FIPS_Code', 'State'])

## Partie clustering : 

In [91]:
import time
import warnings

import numpy as np
import matplotlib.pyplot as plt

from sklearn import cluster, datasets, mixture
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler
from itertools import cycle, islice

np.random.seed(0)

In [92]:
# normalize dataset for easier parameter selection
dataset['Unemployment_rate_2014'] = dataset['Unemployment_rate_2014'].fillna(0)
X = StandardScaler().fit_transform(dataset.values[:,:])
y_cluster = []
compteur = 0

for i in range(2,8,1):
    
    algorithm = cluster.KMeans(init='k-means++', n_clusters=i, max_iter= 600, n_init = 80)
    algorithm.fit(X)
    y_pred = algorithm.predict(X)
    y_cluster.append([])
    y_cluster[compteur].extend(y_pred)
    compteur+=1
    print(compteur)

C:\Users\pfisz010614\AppData\Roaming\Python\Python35\site-packages\sklearn\utils\validation.py:444: DataConversionWarning:

Data with input dtype object was converted to float64 by StandardScaler.



1
2
3
4
5
6


In [93]:
df_y_clustering = pd.DataFrame(data= np.transpose(y_cluster), columns=['nb_cluster_2','nb_cluster_3','nb_cluster_4','nb_cluster_5','nb_cluster_6','nb_cluster_7'])
df_y_clustering.head()

,nb_cluster_2,nb_cluster_3,nb_cluster_4,nb_cluster_5,nb_cluster_6,nb_cluster_7
0,0,2,3,2,2,3
1,0,2,3,2,2,3
2,1,0,0,3,5,0
3,0,1,2,4,4,2
4,1,0,0,4,4,2


In [94]:
df_population = df_population.merge(df_y_clustering, right_index=True, left_index=True)

In [29]:
df_population

,Year_x,State_postal_abbreviation,FIPS_code,American Indian/Alaska Native|Female|0 - 20 Years,American Indian/Alaska Native|Female|20 - 45 Years,American Indian/Alaska Native|Female|45 - 65 Years,American Indian/Alaska Native|Female|65 - 85 Years,American Indian/Alaska Native|Female|85+ Years,American Indian/Alaska Native|Male|0 - 20 Years,American Indian/Alaska Native|Male|20 - 45 Years,...,State,Unemployment_rate_2014,Year_y,Population_Totale,nb_cluster_2,nb_cluster_3,nb_cluster_4,nb_cluster_5,nb_cluster_6,nb_cluster_7
0,2014,AK,02013,0.032464,0.031553,0.026699,0.012136,0.001214,0.035498,0.036711,...,AK,0.045,2014,3296.0,0,0,3,0,5,5
1,2014,AK,02016,0.019723,0.021972,0.017474,0.006920,0.000519,0.020761,0.030450,...,AK,0.048,2014,5780.0,0,0,3,0,5,5
2,2014,AK,02020,0.017169,0.019372,0.011294,0.003515,0.000313,0.017938,0.017665,...,AK,0.052,2014,300429.0,1,1,0,0,5,1
3,2014,AK,02050,0.174397,0.134640,0.075283,0.027340,0.002060,0.183585,0.144440,...,AK,0.161,2014,17959.0,0,2,1,4,4,2
4,2014,AK,02060,0.060510,0.053079,0.072187,0.025478,0.003185,0.061571,0.073248,...,AK,0.080,2014,942.0,0,2,1,4,4,2
5,2014,AK,02068,0.002622,0.003671,0.006817,0.004195,0.000000,0.007341,0.009439,...,AK,0.103,2014,1907.0,1,1,0,0,1,1
6,2014,AK,02070,0.142714,0.119864,0.084987,0.026258,0.002606,0.150732,0.123071,...,AK,0.100,2014,4989.0,0,2,1,4,4,2
7,2014,AK,02090,0.015811,0.017242,0.009578,0.003041,0.000363,0.016960,0.017685,...,AK,0.058,2014,99295.0,1,1,0,0,1,1
8,2014,AK,02100,0.016949,0.020497,0.021285,0.006307,0.000394,0.019314,0.016161,...,AK,0.107,2014,2537.0,1,1,0,2,0,3
9,2014,AK,02105,0.058145,0.059106,0.062470,0.025469,0.002403,0.060067,0.073042,...,AK,0.151,2014,2081.0,0,2,1,4,4,2


## Partie ACP : 

In [96]:
from sklearn.decomposition import PCA
X = StandardScaler().fit_transform(dataset.values[:,:]) 
pca = PCA(n_components=32 )
array = pca.fit_transform(X)

C:\Users\pfisz010614\AppData\Roaming\Python\Python35\site-packages\sklearn\utils\validation.py:444: DataConversionWarning:

Data with input dtype object was converted to float64 by StandardScaler.



In [97]:
pca.explained_variance_ratio_

array([3.70279453e-01, 2.49342042e-01, 8.95201283e-02, 7.46285971e-02,
       6.21441804e-02, 2.98188738e-02, 2.37689621e-02, 2.08258718e-02,
       1.81952457e-02, 1.51632417e-02, 1.03519603e-02, 9.06896510e-03,
       5.94471891e-03, 5.27350211e-03, 4.80669139e-03, 2.84148513e-03,
       1.70887206e-03, 1.32916026e-03, 1.04697286e-03, 8.38507801e-04,
       5.83803828e-04, 4.77908006e-04, 4.66313867e-04, 4.17423294e-04,
       3.17219594e-04, 2.38399286e-04, 2.27184855e-04, 1.51750293e-04,
       1.15726907e-04, 1.06838206e-04, 6.42591096e-32, 8.88835274e-33])

In [98]:
columns = ['Component_1',
       'Component_2',
       'Component_3',
       'Component_4',
       'Component_5',
       'Component_6',
       'Component_7',
       'Component_8',
       'Component_9',
       'Component_10',
       'Component_11',
       'Component_12',
       'Component_13',
       'Component_14',
       'Component_15',
       'Component_16',
       'Component_17',
       'Component_18',
       'Component_19',
       'Component_20',
       'Component_21',
       'Component_22',
       'Component_23',
       'Component_24',
       'Component_25',
       'Component_26',
       'Component_27',
       'Component_28',
       'Component_29',
       'Component_30',
       'Component_31',
       'Component_32']

In [99]:
PCA_Graphique = pd.DataFrame(array, columns = columns)

In [100]:
PCA_Graphique = PCA_Graphique.merge(df_y_clustering, right_index=True, left_index=True)

## Visualisation graphique du CLustering

In [101]:
import plotly.plotly as py
import plotly.graph_objs as go

# Create random data with numpy
import numpy as np

cluster_1 =  PCA_Graphique[PCA_Graphique.nb_cluster_5 == 0]
cluster_2 =  PCA_Graphique[PCA_Graphique.nb_cluster_5 == 1]
cluster_3 =  PCA_Graphique[PCA_Graphique.nb_cluster_5 == 2]
cluster_4 =  PCA_Graphique[PCA_Graphique.nb_cluster_5 == 3]
cluster_5 =  PCA_Graphique[PCA_Graphique.nb_cluster_5 == 4]

# Create a trace
trace_1 = go.Scatter3d(
    x = cluster_1['Component_1'],
    y = cluster_1['Component_2'],
    z = cluster_1['Component_3'],
    mode = 'markers'
)
trace_2 = go.Scatter3d(
    x = cluster_2['Component_1'],
    y = cluster_2['Component_2'],
    z = cluster_2['Component_3'],
    mode = 'markers'
)

trace_3 = go.Scatter3d(
    x = cluster_3['Component_1'],
    y = cluster_3['Component_2'],
    z = cluster_3['Component_3'],
    mode = 'markers'
)

trace_4 = go.Scatter3d(
    x = cluster_4['Component_1'],
    y = cluster_4['Component_2'],
    z = cluster_4['Component_3'],
    mode = 'markers'
)

trace_5 = go.Scatter3d(
    x = cluster_5['Component_1'],
    y = cluster_5['Component_2'],
    z = cluster_5['Component_3'],
    mode = 'markers'
)

data = [trace_1,trace_2,trace_3,trace_4,trace_5] #,trace_4


# Plot and embed in ipython notebook!
py.plotly.tools.set_credentials_file(username='popo95', api_key='UV0tMEa0DGHmbljqHBZz')
py.iplot(data, filename='basic-scatter')

A vous maintenant

In [ ]:
# Lecture des fichiers qui seront utile pour la deuxième question
df_cluster_US_Population = pd.read_csv('D:/Formation/Dauphine/TP1/Finance.csv', encoding='latin-1', sep=',', decimal='.')
df_FIPS = pd.read_csv('D:/Formation/Dauphine/TP1/national_county.txt', index_col=None,sep=',',names=['State_Ab','State_code','County_code','County_name','Other'], dtype=str)
df_cluster_US_Population = df_cluster_US_Population.merge(df_FIPS, left_on = ['State_ab','County'], right_on = ['State_Ab','County_name'], how='inner')
df_cluster_US_Population['FIPS_code'] = df_cluster_US_Population['State_code'] + df_cluster_US_Population['County_code']
df_agg_US_FIPS = pd.pivot_table(df_cluster_US_Population, 
                                 values=['family_income_mean','gross_rent_mean','morgages_ocsts_mean', 'owner_cost_mean', 'household_income_mean'], 
                                 index = ['State_Code', 'State_Name', 'State_ab', 'FIPS_code'], 
                                 aggfunc=[np.mean])
df_agg_US_FIPS.columns = ["|".join((j,k)) for j,k in df_agg_US_FIPS.columns]
df_agg_US_FIPS = df_agg_US_FIPS.reset_index()
df_agg_US_FIPS.head()


In [127]:
# fonction pour un modèle à 2 classe si vous changer les paramètre de np.arange vous pouvez changer le nombre de classe
def search_for_key(value,dict_quantile):
    min_val = 999999999
    val_to_return = 1
    for quantile in dict_quantile.keys():
        if abs(value - dict_quantile[quantile])<min_val:
            min_val = abs(value - dict_quantile[quantile])
            val_to_return = quantile
    return val_to_return
list_y = []
def quantile_dictionary(y):
    dict_quantile = {}
    for value in np.arange(0.0,1.1,0.5):
        dict_quantile[value] = y.quantile(value)
    return dict_quantile
def classe_appartenance(y):
    dict_quantile = quantile_dictionary(y)
    for value in y:
        list_y.append(search_for_key(value,dict_quantile) * 2)
    y_quantile = pd.DataFrame(data = list_y)
    return y_quantile

In [124]:
y_to_pred = df_agg_US_FIPS[['FIPS_code', 'mean|family_income_mean']]
df_classe = classe_appartenance(y_to_pred['mean|family_income_mean'])

In [125]:
y_to_pred = y_to_pred.merge(df_classe, right_index=True, left_index=True, how='inner')

In [126]:
y_to_pred

,FIPS_code,mean|family_income_mean,0
0,01001,61948.946839,1.0
1,01003,66222.833333,1.0
2,01005,50973.000000,1.0
3,01007,57560.500000,1.0
4,01009,69941.000000,1.0
5,01011,75769.000000,1.0
6,01013,50313.000000,1.0
7,01015,60747.000000,1.0
8,01017,38926.500000,1.0
9,01019,0.000000,0.0
